In [0]:
import sys
import os

# 1. Configura Path para importar 'src'
repo_root = os.getcwd()
repo_root = '/'.join(repo_root.split('/')[:-1])
if repo_root not in sys.path:
    sys.path.append(repo_root)



In [0]:
%pip install -r ../requirements.txt

In [0]:
from src.utility.environment import Environment
import mlflow
from pyspark.sql.functions import col, when
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as F

In [0]:
model_uri = f"models:/workspace.kaggle.churn_predictor@production"
print(f"🔄 Carregando modelo de Produção: {model_uri}")
model = mlflow.spark.load_model(model_uri)

In [0]:
# 1. Carregar Dados "Novos"
# Na vida real, filtrariamos apenas clientes ativos que não foram processados no período anterior
#
print("📅 Lendo base de clientes ativos...")
df_gold = spark.read.format("delta").load(f"{Environment.feature_store_path}/churn")

print("🔮 Aplicando modelo de Churn...")
df_scored = model.transform(df_gold)
df_scored = df_scored.withColumn(
            "risk_status",
            when(col("prediction") == 1, "🔴 HIGH RISK").otherwise("🟢 LOW RISK")
        )

# 4. Regra de Negócio Pós-Modelo (A camada de Aplicação)
# O modelo dá a probabilidade. O Negócio define o que fazer com ela.
df_final_report = df_scored\
    .filter(F.col("prediction") == 1)\
    .select(
        "customer_unique_id", 
        "recency", 
        "frequency", 
        "monetary",
        'total_items_volume', 
        'payment_method_count', 
        'max_installments', 
        'avg_satisfaction', 
        'min_review_score', 
        'total_reviews_given',
        "probability", # Importante mostrar a confiança do modelo
        "risk_status"
    )\
    .orderBy(F.col("probability").desc()) # Priorizar os casos mais graves




In [0]:
# PODERIA SALVAR APENAS A PROBABILIDADE DO CHURN, MAS MANTIVE O DENSE VECTOR DE SAIDA DO MODELO
#df_final_report = df_final_report.withColumn(
#    "probability", 
#    vector_to_array(col("probability"))[1]
#)

In [0]:
target_path = f"{Environment.DATABRICKS_CATALOG}/reports"
if not os.path.exists(target_path):
    os.mkdir(target_path)

df_final_report.write\
    .format("delta")\
    .mode("overwrite")\
    .option("overwriteSchema", "true")\
    .saveAsTable("kaggle.churn_daily_report")

print(f"✅ Relatório gerado com sucesso em: kaggle/churn_daily_report")
display(df_final_report.limit(10))